In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('practice-347601-6e0b822fd842.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

#date = 0 #当日は0、前日は1・・・
#
date = 1 #当日は0、前日は1・・・
date2 = "08/08"
pmc = 13197016 # 店舗ID
SPREADSHEET_KEY = '1w1J0evuXmamCckWabee9rkmkVYRJs5StH_CAam6P938' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 549 # スロットの1台目
wait = 0.01

user_id = "g84909252+003@gmail.com"
user_pw = "sitega22"

Template_p_path = "site777/template_p.png"
Template_s_path = "site777/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Program Files\\Google\\Chrome\\Application\\Chrome.exe'
path = "C:\\Users\\garden026\\WD\\" + str(wd) + "\\chromedriver.exe"

chrome_service = ChromeService(executable_path=path)

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "500":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "500":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

time.sleep(5)

SPREADSHEET_KEY = '1zrmSGw05GZ31rA9MY3fdeN8jrrxFlWfTL4a2ovRHckI' # スプレッドシートのID
gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value

time.sleep(5)

wb.values_append("'P4RAW'!B" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!B" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!B" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!B" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['08/08', '025865', '1', '31795.0', '-6619.0', '64', '13']
['08/08', '025659', '2', '33333.0', '1506.0', '40', '14']
['08/08', '025831', '3', '21795.0', '-902.0', '23', '6']
['08/08', '025466', '4', '30000.0', '-3911.0', '32', '12']
['08/08', '025465', '5', '26410.0', '-10832.0', '19', '14']
['08/08', '025572', '6', '38205.0', '-2106.0', '40', '16']
['08/08', '025605', '7', '31538.0', '-7823.0', '22', '12']
['08/08', '025863', '8', '38974.0', '6922.0', '69', '19']
['08/08', '025404', '9', '26154.0', '6621.0', '43', '8']
['08/08', '025404', '10', '20000.0', '-902.0', '23', '6']
['08/08', '025605', '11', '17692.0', '-902.0', '19', '6']
['08/08', '025605', '12', '27692.0', '-8124.0', '19', '10']
['08/08', '025364', '13', '39487.0', '8728.0', '60', '20']
['08/08', '025364', '14', '27179.0', '4515.0', '41', '14']
['08/08', '025521', '15', '34872.0', '4515.0', '39', '14']
['08/08', '025533', '16', '33590.0', '-6920.0', '32', '12']
['08/08', '025689', '17', '32821.0', '-1203.0', '37', '17']
[

['08/08', '025849', '141', '20256.0', '-5717.0', '8', '4']
['08/08', '025849', '142', '8718.0', '4214.0', '8', '1']
['08/08', '025849', '143', '22564.0', '8427.0', '19', '3']
['08/08', '025859', '144', '40000.0', '-17453.0', '18', '5']
['08/08', '025859', '145', '36923.0', '-7221.0', '32', '6']
['08/08', '025825', '146', '26667.0', '-4212.0', '12', '1']
['08/08', '025825', '147', '20000.0', '-7823.0', '7', '3']
['08/08', '025825', '148', '20769.0', '9330.0', '18', '2']
['08/08', '025643', '149', '24615.0', '-13240.0', '5', '4']
['08/08', '025643', '150', '19487.0', '-15346.0', '1', '1']
['08/08', '025829', '151', '21282.0', '-11133.0', '5', '5']
['08/08', '025829', '152', '24359.0', '-4513.0', '11', '3']
['08/08', '025687', '153', '25641.0', '-902.0', '18', '3']
['08/08', '025764', '154', '23590.0', '-14143.0', '4', '1']
['08/08', '025656', '155', '29744.0', '-16851.0', '12', '6']
['08/08', '025836', '156', '37436.0', '11436.0', '30', '6']
['08/08', '025817', '157', '34103.0', '2107.0'

['08/08', '025839', '281', '11282.0', '-10531.0', '0', '0']
['08/08', '025839', '282', '8718.0', '-7823.0', '0', '0']
['08/08', '025839', '283', '7179.0', '302.0', '6', '1']
['08/08', '025839', '284', '12051.0', '12339.0', '20', '4']
['08/08', '025839', '285', '13333.0', '-10531.0', '2', '1']
['08/08', '025643', '286', '16154.0', '-10832.0', '2', '2']
['08/08', '025643', '287', '17949.0', '-12638.0', '2', '2']
['08/08', '025643', '288', '9744.0', '19862.0', '18', '2']
['08/08', '025850', '289', '5897.0', '10232.0', '10', '1']
['08/08', '025850', '290', '5897.0', '-902.0', '3', '1']
['08/08', '025850', '291', '3590.0', '6621.0', '7', '2']
['08/08', '025850', '292', '9487.0', '-6920.0', '1', '1']
['08/08', '025779', '293', '7179.0', '11135.0', '20', '4']
['08/08', '025779', '294', '6154.0', '-5416.0', '1', '1']
['08/08', '025779', '295', '6410.0', '3010.0', '10', '3']
['08/08', '025779', '296', '6667.0', '302.0', '7', '2']
['08/08', '025878', '297', '16410.0', '-10231.0', '8', '2']
['08/

['08/08', '025665', '421', '46923.0', '-27383.0', '11', '6']
['08/08', '025665', '422', '36667.0', '6621.0', '26', '6']
['08/08', '025665', '423', '35385.0', '-10231.0', '16', '6']
['08/08', '025665', '424', '40769.0', '17756.0', '37', '7']
['08/08', '025665', '425', '38462.0', '1205.0', '23', '4']
['08/08', '025665', '426', '39231.0', '1806.0', '24', '3']
['08/08', '025665', '427', '38205.0', '24978.0', '37', '5']
['08/08', '025665', '428', '35385.0', '-26781.0', '5', '4']
['08/08', '025665', '429', '35385.0', '-14744.0', '12', '5']
['08/08', '025665', '430', '37949.0', '22269.0', '36', '6']
['08/08', '025665', '431', '40000.0', '-4814.0', '20', '5']
['08/08', '025665', '432', '38462.0', '-4212.0', '20', '5']
['08/08', '025888', '433', '40513.0', '-15647.0', '12', '7']
['08/08', '025888', '434', '24615.0', '-3610.0', '11', '3']
['08/08', '025888', '435', '31538.0', '-3911.0', '15', '6']
['08/08', '025888', '436', '42308.0', '6621.0', '28', '9']
['08/08', '025888', '437', '29487.0', '3

['08/08', '120010', '559', '7647', '602.0', '30', '22', '--']
['08/08', '120010', '560', '2342', '-324.0', '8', '5', '--']
['08/08', '120010', '561', '5379', '-139.0', '23', '14', '--']
['08/08', '120010', '562', '7240', '-417.0', '23', '29', '--']
['08/08', '120010', '563', '5416', '-370.0', '20', '16', '--']
['08/08', '120010', '564', '2677', '-1065.0', '7', '3', '--']
['08/08', '120010', '565', '4777', '1157.0', '23', '11', '--']
['08/08', '120010', '566', '5606', '1019.0', '24', '19', '--']
['08/08', '120010', '567', '1680', '-694.0', '5', '2', '--']
['08/08', '120010', '568', '6142', '-880.0', '19', '12', '--']
['08/08', '120010', '569', '4995', '-93.0', '18', '15', '--']
['08/08', '120010', '570', '8308', '556.0', '34', '24', '--']
['08/08', '120010', '571', '9648', '787.0', '36', '33', '--']
['08/08', '120010', '572', '4909', '-370.0', '17', '14', '--']
['08/08', '120010', '573', '5513', '139.0', '18', '16', '--']
['08/08', '120010', '574', '5500', '417.0', '25', '10', '--']
['0

['08/08', '119916', '693', '4444', '741.0', '17', '19', '--']
['08/08', '119916', '694', '3031', '231.0', '14', '3', '--']
['08/08', '119916', '695', '2605', '-880.0', '5', '11', '--']
['08/08', '119916', '696', '3654', '-602.0', '10', '11', '--']
['08/08', '119916', '697', '5078', '-556.0', '18', '13', '--']
['08/08', '119916', '698', '6926', '2731.0', '31', '30', '--']
['08/08', '119916', '699', '2837', '-278.0', '7', '12', '--']
['08/08', '119916', '700', '5260', '-324.0', '19', '13', '--']
['08/08', '119916', '701', '943', '-139.0', '3', '1', '--']
['08/08', '119916', '702', '5417', '787.0', '24', '13', '--']
['08/08', '119916', '703', '733', '-463.0', '2', '0', '--']
['08/08', '119916', '704', '6013', '880.0', '24', '20', '--']
['08/08', '119916', '705', '2204', '-93.0', '8', '8', '--']
['08/08', '119916', '706', '6585', '1574.0', '31', '21', '--']
['08/08', '119916', '707', '5187', '509.0', '22', '11', '--']
['08/08', '119916', '708', '5047', '-93.0', '19', '13', '--']
['08/08', 

['08/08', '120145', '826', '6893', '-556.0', '0', '11', '58']
['08/08', '120145', '827', '7127', '4630.0', '0', '10', '100']
['08/08', '120145', '828', '7584', '-3935.0', '0', '18', '49']
['08/08', '120104', '829', '6418', '-2315.0', '0', '8', '26']
['08/08', '120103', '830', '2433', '-1898.0', '7', '3', '10']
['08/08', '120138', '831', '6320', '5509.0', '0', '16', '40']
['08/08', '120103', '832', '2874', '-2917.0', '4', '2', '12']
['08/08', '120126', '833', '5403', '-463.0', '0', '13', '31']
['08/08', '120034', '834', '1482', '-185.0', '0', '3', '13']
['08/08', '120139', '835', '5224', '1620.0', '0', '11', '37']
['08/08', '120121', '836', '3287', '231.0', '0', '5', '18']
['08/08', '120132', '837', '1327', '972.0', '13', '2', '--']
['08/08', '120126', '838', '7461', '602.0', '0', '15', '34']
['08/08', '120053', '839', '3981', '324.0', '0', '10', '60']
['08/08', '119937', '840', '3553', '2731.0', '0', '1', '93']
['08/08', '119955', '841', '7368', '741.0', '0', '7', '84']
['08/08', '1201

['08/08', '120034', '960', '1567', '-694.0', '0', '3', '10']
['08/08', '120034', '961', '584', '-648.0', '0', '0', '2']
['08/08', '120034', '962', '2287', '-185.0', '0', '3', '17']
['08/08', '120034', '963', '7287', '1713.0', '0', '12', '73']
['08/08', '120096', '964', '1791', '-556.0', '5', '5', '--']
['08/08', '120096', '965', '874', '-556.0', '2', '0', '--']
['08/08', '119945', '966', '1938', '-231.0', '8', '3', '--']
['08/08', '120117', '967', '1989', '-694.0', '0', '4', '29']
['08/08', '119926', '968', '5230', '1343.0', '0', '7', '68']
['08/08', '120121', '969', '131', '-139.0', '0', '0', '--']
['08/08', '119955', '970', '4497', '2500.0', '0', '6', '80']
['08/08', '120146', '971', '2088', '-1898.0', '0', '4', '2']
['08/08', '120146', '972', '3279', '-880.0', '0', '6', '7']
['08/08', '120065', '973', '4339', '556.0', '14', '16', '--']
['08/08', '120036', '974', '2972', '-556.0', '8', '6', '--']
['08/08', '120036', '975', '4967', '139.0', '18', '11', '--']
['08/08', '120137', '976',

{'spreadsheetId': '1zrmSGw05GZ31rA9MY3fdeN8jrrxFlWfTL4a2ovRHckI',
 'tableRange': 'S5RAW!A1:J11245',
 'updates': {'spreadsheetId': '1zrmSGw05GZ31rA9MY3fdeN8jrrxFlWfTL4a2ovRHckI',
  'updatedRange': 'S5RAW!A11246:H11282',
  'updatedRows': 37,
  'updatedColumns': 8,
  'updatedCells': 296}}

In [2]:
browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "500":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "500":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

time.sleep(5)

SPREADSHEET_KEY = '1zrmSGw05GZ31rA9MY3fdeN8jrrxFlWfTL4a2ovRHckI' # スプレッドシートのID
gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value

time.sleep(5)

wb.values_append("'P4RAW'!B" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!B" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!B" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!B" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

KeyboardInterrupt: 